In [179]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import matplotlib
from numpy import linalg as LA
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms, datasets
import torch.optim as optim
from PIL import Image
import os

In [84]:
class sparse_autoencoder(torch.nn.Module):

    def __init__(self):
        super(sparse_autoencoder, self).__init__()
        input_size = 784
        output_size = 784      
        self.hidden = nn.Linear(input_size, 256)
        self.out = nn.Linear(256, output_size)
        self.batch_size = 8

    def forward(self, x):
        h = F.sigmoid(self.hidden(x))
        y_hat = self.out(h)
        return y_hat, h
    
    def loss(self, x, y, beta=0.1, rho=0.05):
        y_hat, h = self.forward(x)
        rho_hat = torch.sum(h, dim=0) / self.batch_size
        c = nn.MSELoss()
        l = c(y_hat, y) + beta * torch.sum(rho * torch.log(rho / rho_hat) + (1-rho) * torch.log((1-rho) / (1-rho_hat)))
        return l

In [85]:
ae = sparse_autoencoder()
optimizer = optim.Adam(ae.parameters(), lr=3e-4)

In [86]:
batch_size = 8
train = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
trainset = torch.utils.data.DataLoader(train, batch_size=batch_size, shuffle=True)

In [91]:
train
trans = transforms.ToPILImage()

In [100]:
for epoch in range(3):
    cnt = 0
    l = 0
    for data in trainset:
        data = data[0].squeeze()
        x = torch.reshape(data, (batch_size, 784))
        y = x.clone()
        optimizer.zero_grad()
        y_hat, h = ae(x)
        loss = ae.loss(x, y)
        loss.backward()
        optimizer.step()
        l += loss
        cnt += 1
        if cnt % 600 == 0:
            cnt = 0
            print(l / 600)
            l = 0

tensor(0.1001, grad_fn=<DivBackward0>)
tensor(0.0934, grad_fn=<DivBackward0>)
tensor(0.0904, grad_fn=<DivBackward0>)
tensor(0.0928, grad_fn=<DivBackward0>)
tensor(0.0914, grad_fn=<DivBackward0>)
tensor(0.0864, grad_fn=<DivBackward0>)
tensor(0.0902, grad_fn=<DivBackward0>)
tensor(0.0822, grad_fn=<DivBackward0>)
tensor(0.0843, grad_fn=<DivBackward0>)
tensor(0.0867, grad_fn=<DivBackward0>)
tensor(0.0807, grad_fn=<DivBackward0>)
tensor(0.0799, grad_fn=<DivBackward0>)
tensor(0.0771, grad_fn=<DivBackward0>)
tensor(0.0781, grad_fn=<DivBackward0>)
tensor(0.0745, grad_fn=<DivBackward0>)
tensor(0.0757, grad_fn=<DivBackward0>)
tensor(0.0715, grad_fn=<DivBackward0>)
tensor(0.0703, grad_fn=<DivBackward0>)
tensor(0.0727, grad_fn=<DivBackward0>)
tensor(0.0673, grad_fn=<DivBackward0>)
tensor(0.0674, grad_fn=<DivBackward0>)
tensor(0.0690, grad_fn=<DivBackward0>)
tensor(0.0695, grad_fn=<DivBackward0>)
tensor(0.0688, grad_fn=<DivBackward0>)
tensor(0.0640, grad_fn=<DivBackward0>)
tensor(0.0630, grad_fn=<D

In [113]:
test = datasets.MNIST(root='./data', train=False, download=True, transform=transforms.Compose([transforms.ToTensor()]))
testset = torch.utils.data.DataLoader(test, batch_size=batch_size, shuffle=True)

In [181]:
results = []
cnt = 0
os.makedirs("img", exist_ok=True)
for test in testset:
    data = test[0].squeeze()
    x = torch.reshape(data, (batch_size, 784))
    out = ae(x)[0].detach().numpy()
    cnt += 1
    for i in range(data.shape[0]):
        plt.imsave('./img/' + str(cnt) + str(i) + 'org.png', data[i], cmap='gray')
        plt.imsave('./img/' + str(cnt) + str(i) + 'recovered.png', out[i].reshape(28, 28), cmap='gray')